<a href="https://colab.research.google.com/github/bryanf98/IA_2020B/blob/main/SVM/Clasificacion_de_imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clasificación de imagenes**

Escuela Politécnica Nacional

Inteligencia Artificial

Alumno: Bryan Flores Sarango

Fecha:09-02-2021

In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split

from skimage.io import imread
from skimage.transform import resize

In [2]:
from google.colab import drive
#os.chdir("/content/drive/My Drive")
drive.mount('/content/drive/')

Mounted at /content/drive/


**Cargue imágenes en un directorio estructurado como si fuera un conjunto de datos de muestra de sklearn**

In [3]:
ls "/content/drive/My Drive"

'20180322_173616 1.mp4'
'Colab Notebooks'/
'Google Fotos'/
 Grupo3_Teoria_Invision.mp4
 images/
'Inteligencia Artificial'/
'Mecanica para ingenieros.pdf'
'Mecanica Vectorial para ingenieros Estatica - Edicion 9 - Beer, Johnston.pdf'
'Tecnologías de Seguridad'/
 testeo/


In [4]:
cp -r "/content/drive/My Drive/images/" "images"

In [5]:
ls 

drive/  images/  sample_data/


**Validamos con PIL**

In [ ]:
import os
from os import listdir
from PIL import Image

dir_path = "images"

for dirName, subdirList, fileList in os.walk(dir_path):
    #print('Directorio encontrado: %s' % dirName)
    for fname in fileList:
          try:
            if fname.endswith('.jpg'):
              img = Image.open(dir_path+"\\"+fname)
              if(img.verify()==True):
                print("1 ",fname)
            else:
              print("2")
          except (IOError, SyntaxError) as e:
            print("")
              

In [7]:
def load_image_files(container_path, dimension=(64, 64)):
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "Conjunto de datos de clasificación de imágenes"
    images = []
    flat_data = []
    target = []
    
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = imread(file)
            img_resized = resize(img, dimension, anti_aliasing=True, mode='reflect')
            flat_data.append(img_resized.flatten()) 
            images.append(img_resized)
            target.append(i)
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)

**Cargamos el dataset**

In [32]:
image_dataset = load_image_files("images")

**Dividimos los datos**

In [33]:
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.3,random_state=109)

**Entrene datos con optimización de parámetros**

In [34]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
svc = svm.SVC()
svc.fit(X_train, y_train)
clf = GridSearchCV(svc, param_grid)
clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

**Predicción**

In [28]:
y_pred = svc.predict(X_test)

**Reporte**

In [35]:
print("Reporte de clasificación - \n{}:\n{}\n".format(
    svc, metrics.classification_report(y_test, y_pred)))

Reporte de clasificación - 
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False):
              precision    recall  f1-score   support

           0       0.74      0.88      0.80        16
           1       0.93      0.96      0.95        27
           2       0.73      0.57      0.64        14
           3       0.67      0.76      0.71        21
           4       0.91      0.67      0.77        15

    accuracy                           0.80        93
   macro avg       0.79      0.77      0.77        93
weighted avg       0.80      0.80      0.79        93




**Predicción**

In [26]:
y_pred = clf.predict(X_test)

**Reporte**

In [36]:
print("Reporte de clasificación - \n{}:\n{}\n".format(
    clf, metrics.classification_report(y_test, y_pred)))

Reporte de clasificación - 
GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0):
              precision    recall  f1-score   support

           0       0.74      0.88      0.80        16
           1       0.93      0.96      0.95        27
           2    

**Guardar el modelo**

In [ ]:
import pickle
filename='modeloSVC.h5'
pickle.dump(svc, open(filename, 'wb'))

In [ ]:
from google.colab import files
download = files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Bibliografia: https://github.com/whimian/SVM-Image-Classification